In [129]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/gathering_affinities.ipynb")

Populating the interactive namespace from numpy and matplotlib


In [130]:
# get proteome reference
merged = gather_protein_sequences()

In [135]:
merged.head()

,gene,protein,sequence,uniprot_AC,length
99474,TTN,ENSP00000467141,MTTQAPTFTQPLQSVVVLEGSTATFEAHISGFPVPEVSWFRDGQVI...,A0A0A0MTS7,35991
99472,TTN,ENSP00000343764,MTTQAPTFTQPLQSVVVLEGSTATFEAHISGFPVPEVSWFRDGQVI...,Q8WZ42,33423
99477,TTN,ENSP00000340554,MTTQAPTFTQPLQSVVVLEGSTATFEAHISGFPVPEVSWFRDGQVI...,A0A0A0MRA3,27118
72694,MUC16,ENSP00000381008,MLKPSGLPGSSSPTRSLMTGSRSTKATPEMDSGLTGATLSPKTSTG...,Q8WXI7,14507
61505,OBSCN,ENSP00000355668,MDQPQFSGAPRFLTRPKAFVVSVGKDATLSCQIVGNPTPQVSWEKD...,A6NGQ3,8923


In [144]:
# open the raw files
ms = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/validation/ciudad/raw/DonorA_DRB10301_DRB11101.csv')
ms = ms[['Peptide sequence', 'Uniprot AC']]
ms.columns = ['measured_peptide', 'uniprot_AC']
print len(ms)

140


In [146]:
# merge together based on protein id
ms = pd.merge(ms, merged, on='uniprot_AC', how='left')

In [147]:
ms.head()

,measured_peptide,uniprot_AC,gene,protein,sequence,length
0,PSGWWKGRLHGQEGLFPGNYVEKI,O00160,MYO1F,ENSP00000344871,MGSKERFHWQSHNVKQSGVDDMVLLPQITEDAIAANLRKRFMDDYI...,1098.0
1,HVGFIFKNGKITSIVK,O00560,SDCBP,ENSP00000260130,MSLYPSLEDLKVDKVIQAQTAFSANPANPAILSEASAPIPHDGNLY...,298.0
2,ITSIVKDSSAARNGLL,O00560,SDCBP,ENSP00000260130,MSLYPSLEDLKVDKVIQAQTAFSANPANPAILSEASAPIPHDGNLY...,298.0
3,PRVPWVKMILNKLSQ,O00626,CCL22,ENSP00000219235,MDRLQTALLVVLVLLAVALQATEAGPYGANMEDSVCCRDYVRYRLP...,93.0
4,RGFYCNDESIKYPL,O14495,PLPP3,ENSP00000360296,MQNYKYDKAIVPESKNGGSPALNNNPRRSGSKRVLLICLDLFCLFM...,311.0


In [149]:
def get_combined(x):
    try:
        gene = x[0]
        peptide = x[1]
        aa = peptide[len(peptide)/2]
        residue = list(ms[ms.measured_peptide == peptide].sequence)[0].find(peptide) + 1 + (len(peptide) / 2)
        return '{0}_{1}{2}{3}'.format(gene, aa, residue, aa)
    except:
        return 'fail'
ms['combined'] = ms[['gene', 'measured_peptide', 'sequence']].apply(get_combined, axis=1)

In [151]:
processed = ms[ms.combined != 'fail']
mutations = list(set(processed.combined))

In [ ]:
with open('/cellar/users/ramarty/Data/hla_ii/validation/ciudad/residues/{0}.txt'.format(allele), 'w') as outfile:
    for mutation in mutations:
        outfile.write('{0}\n'.format(mutation))